### Chatbot logic and implementation

In [27]:
# Import necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import Tuple, Union, List

# Used for sentiment analysis
from textblob import Blobber
from textblob.sentiments import NaiveBayesAnalyzer

# Used for recommendation system
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
class BookChat:
    """
    A chatbot that can recommend books and provide customer support.

    Attributes:
        books (pandas.DataFrame): A DataFrame containing information about available books.
        orders (pandas.DataFrame): A DataFrame containing information about customer orders.
        customers (pandas.DataFrame): A DataFrame containing information about customers.
        reviews (pandas.DataFrame): A DataFrame containing customer reviews of books.
        tb (textblob.Blobber): A TextBlob object with a NaiveBayesAnalyzer for sentiment analysis.

    Methods:
        reply(msg: str) -> None:
            Prints a message from the chatbot.

        user_msg(msg: str) -> None:
            Prints a message from the user.

        input(msg: str) -> str:
            Prints a message from the chatbot and gets input from the user.

        get_sentiment(text: str) -> str:
            Returns the sentiment of a given text as "positive", "negative", or "neutral".
    """

    def __init__(self):
        """
        Initializes a new BookChat instance.

        Loads data from CSV files and initializes a TextBlob object with a NaiveBayesAnalyzer.
        """
        self.books = pd.read_csv("./data/books.csv")
        self.orders = pd.read_csv("./data/orders.csv")
        self.customers = pd.read_csv("./data/customers.csv")
        self.reviews = pd.read_csv("./data/reviews.csv")

        self.tb = Blobber(analyzer=NaiveBayesAnalyzer())

    def reply(self, msg: str) -> None:
        print(f"\nBookChat: {msg}")

    def user_msg(self, msg: str) -> None:
        print(f"\nUser: {msg}")

    def input(self, msg: str) -> str:
        self.reply(msg)
        user_input = input("\nUser: ")
        # self.user_msg(user_input)  # Only keep in VSCode
        return user_input

    def get_sentiment(self, text: str) -> str:
        sentiment = self.tb(text).sentiment
        return (
            "neutral"
            if sentiment.p_pos == sentiment.p_neg
            else "positive"
            if sentiment.p_pos > sentiment.p_neg
            else "negative"
        )

In [6]:
class Books(BookChat):
    """
    A chatbot that recommends books and provides customer support.

    Inherits from the BookChat class.

    Methods:
        get_book_details(book_title: str) -> pandas.Series:
            Returns a pandas Series containing details about a book with the given title.

        check_availability() -> Union[pandas.Series, List]:
            Asks the user for a book title and checks if it is in stock. Returns a pandas Series with details about the book if it is in stock, or an empty list if it is not.

        check_inventory(book: pandas.Series) -> None:
            Prints the number of copies of a book that are currently in stock.

        check_price(book: pandas.Series) -> None:
            Prints the price of a book.

        generate_cosine_matrix() -> numpy.ndarray:
            Generates a cosine similarity matrix for the `books` DataFrame using the specified columns and TF-IDF algorithm.

        get_recommendations(feature_name: str, feature_query: str, cosine_sim: numpy.ndarray) -> Union[pandas.Series, bool]:
            Returns a pandas Series containing the top 5 books that are most similar to the given feature query, based on the cosine similarity matrix.

        ask_for_recommendation() -> Tuple[str, str]:
            Asks the user if they want a book recommendation by title or genre, and returns a tuple containing the feature name and query.

        show_recommendations(feature_name: str, feature_query: str) -> Union[List[str], None]:
            Generates book recommendations based on the given feature name and query, and prints them to the console.

        query_recommendations(recommendation_list: pandas.Series) -> None:
            Asks the user about the availability and price of a book in a recommended book list.
    """

    def __init__(self):
        super().__init__()

    def get_book_details(self, book_title: str) -> pd.Series:
        return self.books[self.books["title"] == book_title].iloc[0]

    def check_availability(self) -> pd.Series:
        user_input = self.input(msg="What book are you looking for?")
        self.reply(msg="One moment, let me check the inventory.")

        if user_input:
            book = self.books[self.books["title"].str.contains(user_input, case=False)]

            if book.empty:
                self.reply(
                    f"Sorry, it seems that we do not have any book with the title '{user_input}' in stock."
                )

                return []

            else:
                self.reply(
                    f"Yes, '{book.iloc[0]['title']}' by {book.iloc[0]['author']} is currently in stock."
                )

                return book.iloc[0]
        else:
            self.reply("Sorry, I did not catch that. Could you please start again?")
            self.check_availability()

    def check_inventory(self, book: pd.Series) -> None:
        self.user_msg("How many copies are available?")

        self.reply("One moment, let me check the inventory.")

        self.reply(f"We currently have {book['quantityInStock']} copies in stock.")

    def check_price(self, book: pd.Series) -> None:
        self.user_msg("What is the price of the book?")

        self.reply("Let me find that information for you...")

        self.reply(
            f"{book['title']} by {book['author']} is priced at ${book['price']}."
        )

    def generate_cosine_matrix(self) -> np.ndarray:
        # Create a TfidfVectorizer object and combine all columns
        tfidf = TfidfVectorizer(stop_words="english")

        columns_to_combine = [
            "title",
            "author",
            "genre",
            "short_description",
            "publisher",
        ]
        self.books["combined"] = (
            self.books[columns_to_combine]
            .apply(lambda x: " ".join(x.astype(str)), axis=1)
            .str.lower()
        )

        tfidf_matrix = tfidf.fit_transform(self.books["combined"])

        # Compute the cosine similarity matrix
        cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

        return cosine_sim

    # Function that takes in book feature as input and outputs most similar books
    def get_recommendations(
        self, feature_name: str, feature_query: str, cosine_sim: np.ndarray
    ) -> Union[pd.Series, bool]:
        try:
            # Get the index of the book that matches the title or genre
            idx = self.books.loc[
                self.books[feature_name].str.lower() == feature_query.lower()
            ].index[0]

            # Get the pairwsie similarity scores of all books with that book
            sim_scores = list(enumerate(cosine_sim[idx]))

            # Sort the books based on the similarity scores
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

            # Get the scores of the 10 most similar books
            sim_scores = sim_scores[1:11]

            # Get the book indices
            book_indices = [i[0] for i in sim_scores]

            # Return the top 10 most similar books
            return self.books["title"].iloc[book_indices]
        except IndexError:
            print(f"Error: {feature_query} not found in our inventory.")
            return False

    def ask_for_recommendation(self) -> Tuple[str, str]:
        self.user_msg("Can you recommend a book for me?")
        rec_choice = self.input(
            msg="Sure, do you want a recommendation by title or genre? (Enter 'Title' / 'Genre')"
        ).lower()

        if "title" in rec_choice:
            return "title", self.input(
                msg="Can you type the name of the book you want suggestions for?"
            )
        elif "genre" in rec_choice:
            return "genre", self.input(msg="What genre are you interested in?")
        else:
            self.reply("Sorry, I did not catch that. Could you please start again?")
            return self.ask_for_recommendation()

    def show_recommendations(
        self, feature_name: str, feature_query: str
    ) -> Union[List[str], None]:
        cosine_sim = self.generate_cosine_matrix()

        recommended_book = self.get_recommendations(
            feature_name, feature_query, cosine_sim
        )

        if recommended_book is not False:
            rec_list = "\n".join(
                [f"{i+1}. {book}" for i, book in enumerate(recommended_book)]
            )

            self.reply(f"Here are some books you might like: \n{rec_list}.")

            return recommended_book
        else:
            return self.reply(
                "Sorry, I could not find any books that match your preference."
            )

    def query_recommendations(
        self, recommendation_list: pd.Series, recommendation_choice: str
    ) -> None:
        # Check if recommendation choice can be converted to number
        try:
            recommendation_choice = int(recommendation_choice)
        except ValueError:
            self.reply("Sorry, I did not catch that. Could you please start again?")
            return

        self.reply(f"Let me check the inventory for Book #{recommendation_choice}...")

        book_details = self.get_book_details(
            recommendation_list.iloc[recommendation_choice - 1]
        )

        self.reply(
            f"We currently have {book_details['quantityInStock']} copies of {book_details['title']} in stock."
        )

        self.user_msg("What is the price of that book?")
        self.reply("Let me find that information for you...")

        self.reply(f"{book_details['title']} is priced at ${book_details['price']}.")

        return

In [7]:
class Orders(BookChat):
    """
    A chatbot that handles customer orders and provides customer support.

    Inherits from the BookChat class.

    Methods:
        get_order(orderID: int) -> Union[pandas.Series, None]:
            Returns a pandas Series containing details about an order with the given order ID, or None if the order ID is invalid.

        get_orders_for_customer(customerID: int) -> pandas.Series:
            Asks the user for their customer ID and returns a pandas Series containing the order IDs associated with that customer.

        reserve_copy(book: pandas.Series, customer: pandas.Series) -> None:
            Reserves a copy of a book for a customer, updates the orders and books DataFrames, and sends a confirmation message to the customer.

        request_replacement(customerInfo: pandas.Series) -> None:
            Asks the user for their order ID and initiates a book replacement request, sending a confirmation message to the customer.

        add_review(customerInfo: pandas.Series) -> None:
            Asks the user for their order ID and prompts them to provide a book review, updates the reviews DataFrame, and sends a confirmation message to the customer.

        process_reviews() -> pandas.DataFrame:
            Processes the reviews DataFrame by adding a sentiment column and returns a DataFrame with the review ID, rating, text, and sentiment.
    """

    def __init__(self):
        super().__init__()

    def get_order(self, orderID: int) -> Union[pd.Series, None]:
        try:
            return self.orders.loc[self.orders["orderID"] == orderID].iloc[0]
        except IndexError:
            return None

    def get_orders_for_customer(self, customerID: int) -> pd.Series:
        order_ids = self.orders[self.orders["customerID"] == customerID]["orderID"]
        self.reply(f"Your order numbers are: {','.join(map(str, order_ids.tolist()))}")
        return order_ids

    def reserve_copy(self, book: pd.Series, customer: pd.Series) -> None:
        self.user_msg("Can you reserve a copy for me?")

        # Update orders table with new orderID
        new_order = {
            "orderID": self.orders["orderID"].max() + 1,
            "bookID": book["bookID"],
            "customerID": customer["customerID"],
            "orderDate": datetime.now().strftime("%Y-%m-%d"),
            "pickupDate": (datetime.now() + timedelta(days=2)).strftime("%Y-%m-%d"),
            "status": "Reserved",
        }

        self.orders = pd.concat(
            [self.orders, pd.DataFrame([new_order])], ignore_index=True
        )
        self.orders.to_csv("./data/orders_DUMP.csv", index=False)

        # Reduce quantityInStock by 1
        self.books.loc[self.books["bookID"] == book["bookID"], "quantityInStock"] -= 1
        self.books.to_csv("./data/books_DUMP.csv", index=False)

        self.reply(
            f"Thank you {customer['firstName']}, I have reserved a copy of {book['title']} for you. Please visit the store within the next 48 hours to pick it up."
        )

        return

    def request_replacement(self, customerInfo: pd.Series) -> None:
        self.reply(
            f"Sorry to hear that, {customerInfo['firstName']}. I can help you with that."
        )

        order_ids = self.get_orders_for_customer(customerInfo["customerID"])

        orderID = int(self.input("Can you please provide your order ID?"))

        if orderID not in order_ids.tolist():
            self.reply("Sorry, it seems that you have entered an invalid order ID.")
            return self.request_replacement(customerInfo)

        order_details = self.get_order(orderID)

        replaced_book = self.books.loc[
            self.books["bookID"] == order_details["bookID"]
        ].iloc[0]

        return self.reply(
            f"Thank you for your request.\n\nOrder #{orderID}, \"{replaced_book['title']}\" will be replaced within 2-3 working days."
        )

    def add_review(self, customerInfo: pd.Series) -> None:
        # Function to add review to the database for a given orderID and customerID
        self.reply(
            f"Thank you, {customerInfo['firstName']}. Which order would you like to provide feedback for?"
        )
        order_ids = self.get_orders_for_customer(customerInfo["customerID"])
        orderID = int(self.input("Can you please provide your order ID?"))

        if orderID not in order_ids.tolist():
            self.reply("Sorry, it seems that you have entered an invalid order ID.")
            return self.add_review(customerInfo)

        order_details = self.get_order(orderID)

        book_details = self.books.loc[
            self.books["bookID"] == order_details["bookID"]
        ].iloc[0]

        review_rating = int(
            self.input(
                f"How would you rate the book \"{book_details['title']}\"? (on a scale of 1-5)"
            )
        )
        review_text = self.input(f"How was your experience with the book?")

        review_data = {
            "reviewID": self.reviews["reviewID"].max() + 1,
            "orderID": orderID,
            "customerID": customerInfo["customerID"],
            "reviewRating": review_rating,
            "reviewText": review_text,
            "reviewDate": datetime.now().strftime("%Y-%m-%d"),
        }

        self.reviews = pd.concat(
            [self.reviews, pd.DataFrame([review_data])], ignore_index=True
        )

        self.reviews.to_csv("./data/reviews_DUMP.csv", index=False)

        review_sentiment = self.get_sentiment(review_text)

        if review_sentiment == "positive":
            self.reply(
                f"We are glad you enjoyed the book! Thank you for your review, {customerInfo['firstName']}!"
            )
        elif review_sentiment == "negative":
            self.reply(
                f"We are sorry to hear that your experience was not enjoyable, {customerInfo['firstName']}. Thank you for your review."
            )
        else:
            self.reply(f"Thank you for your review, {customerInfo['firstName']}.")

        return

    def process_reviews(self) -> pd.DataFrame:
        self.reply("Processing reviews...")
        self.reviews["reviewSentiment"] = self.reviews["reviewText"].apply(
            self.get_sentiment
        )

        self.reply("Here are the reviews with their sentiments:")
        return self.reviews[
            ["reviewID", "reviewRating", "reviewText", "reviewSentiment"]
        ]

In [8]:
class Customers(BookChat):
    """
    A chatbot that handles customer information and provides customer support.

    Inherits from the BookChat class.

    Methods:
        get_customer(customerID: int) -> Union[pandas.Series, None]:
            Returns a pandas Series containing details about a customer with the given customer ID, or None if the customer ID is invalid.

        add_customer() -> pandas.Series:
            Asks the user for their name, phone number, email, and address, generates a new customer ID, adds the customer to the customers DataFrame, and sends a confirmation message to the customer.
    """

    def __init__(self):
        super().__init__()

    def get_customer(self, customerID: int) -> Union[pd.Series, None]:
        try:
            return self.customers[self.customers["customerID"] == customerID].iloc[0]
        except IndexError:
            return None

    def add_customer(self) -> pd.Series:
        self.reply(
            "Welcome to Argo! We hope you have a great experience with us. To help us assist you better, please provide a few details about yourself."
        )

        # Ask for name, address, phone number, email
        firstName = self.input("What is your first name?")
        lastName = self.input("What is your last name?")
        phone = self.input("What is your phone number?")
        email = self.input("What is your email address? (Leave blank if none)")
        address = self.input("What is your address? (Leave blank if not desired)")

        new_customer_id = self.customers["customerID"].max() + 1

        new_customer = {
            "customerID": new_customer_id,
            "firstName": firstName,
            "lastName": lastName,
            "phone": phone,
            "email": email,
            "address": address,
        }

        self.customers = pd.concat(
            [self.customers, pd.DataFrame([new_customer])], ignore_index=True
        )

        self.customers.to_csv("./data/customers_DUMP.csv", index=False)

        self.reply(
            f"Thank you for shopping with us! Your customer ID is {new_customer_id}."
        )
        self.reply("Please use this customer ID for future purchases and queries.")
        self.reply(f"What can I help you with, {firstName}?")

        return self.get_customer(new_customer_id)

In [9]:
class Store(BookChat):
    """
    A chatbot that provides information about the bookstore.

    Inherits from the BookChat class.

    Methods:
        store_hours() -> None:
            User asks for the store hours and the bot replies with the store hours.

        store_location() -> None:
            User asks for the store location and the bot replies with the store location.

        areas_served() -> None:
            User asks for what areas the store serves and the bot replies with the areas served.

        delivery() -> None:
            User asks if the store delivers books and the bot replies with the delivery information.
    """

    def __init__(self):
        super().__init__()

    def store_hours(self) -> None:
        self.user_msg("What are your store hours?")
        self.reply(
            "Our store is open from 11 AM to 7 PM from Monday to Friday. On Saturdays and Sundays, we are open from 12 PM to 5 PM."
        )

    def store_location(self) -> None:
        self.user_msg("Where is your store located?")
        self.reply(
            "We are located at 1841-A Saint-Catherine St W, Montreal, QC H3H 1M2."
        )

    def areas_served(self) -> None:
        self.user_msg("What areas do you serve?")
        self.reply(
            "We have only one store in Montreal, but we ship anywhere in the Island of Montreal, as well as Laval and South Store."
        )

    def delivery(self) -> None:
        self.user_msg("Do you deliver books?")
        self.reply(
            "Yes, we do deliver books in the Island of Montreal. Local delivery is a flat rate of $8.50 (plus tax), but it's free for any purchase of $100 or more."
        )

In [10]:
# Initializing the instances of the classes
chatbot = BookChat()
books = Books()
orders = Orders()
customers = Customers()
store = Store()

### Welcome

##### Customer info - ask if new or returning

In [11]:
chatbot.reply("Hello, I'm BookChat by Argo Bookstore!")

customer_type = chatbot.input(
    "Are you a new customer or an existing customer? (New/Existing)"
).lower()

if "existing" in customer_type:
    while True:
        customerInfo = customers.get_customer(
            int(
                chatbot.input(
                    "Welcome back! Please provide your customer ID so that we can provide you with a personalized experience!"
                )
            )
        )

        if customerInfo is not None:
            chatbot.reply(f"Welcome back, {customerInfo['firstName']}! What can I help you with?")
            break

        else:
            chatbot.reply("Sorry, I couldn't find your customer ID. Please try again.")

elif "new" in customer_type:
    customerInfo = customers.add_customer()

else:
    chatbot.reply("Sorry, I didn't understand that.")


BookChat: Hello, I'm BookChat by Argo Bookstore!

BookChat: Are you a new customer or an existing customer? (New/Existing)

BookChat: Welcome to Argo! We hope you have a great experience with us. To help us assist you better, please provide a few details about yourself.

BookChat: What is your first name?

BookChat: What is your last name?

BookChat: What is your phone number?

BookChat: What is your email address? (Leave blank if none)

BookChat: What is your address? (Leave blank if not desired)

BookChat: Thank you for shopping with us! Your customer ID is 1021.

BookChat: Please use this customer ID for future purchases and queries.

BookChat: What can I help you with, Lakshya?


### Scenario 1: Book availablity and booking

##### 1.1. Check availability of a book

In [12]:
bookSearched = books.check_availability()


BookChat: What book are you looking for?

BookChat: One moment, let me check the inventory.

BookChat: Yes, 'A Farewell to Arms' by Ernest Hemingway is currently in stock.


##### 1.2. Check inventory for selected book

In [13]:
books.check_inventory(bookSearched)


User: How many copies are available?

BookChat: One moment, let me check the inventory.

BookChat: We currently have 65 copies in stock.


##### 1.3. Check price of selected book

In [14]:
books.check_price(bookSearched)


User: What is the price of the book?

BookChat: Let me find that information for you...

BookChat: A Farewell to Arms by Ernest Hemingway is priced at $10.99.


##### 1.4. Place an order for the selected book

In [15]:
orders.reserve_copy(bookSearched, customerInfo)


User: Can you reserve a copy for me?

BookChat: Thank you Lakshya, I have reserved a copy of A Farewell to Arms for you. Please visit the store within the next 48 hours to pick it up.


### Scenario 2: Book Recommendation

##### 2.1. Ask user for recommendation type

In [16]:
# Recommend by title or by genre, depending on user input
recommend_question = books.ask_for_recommendation()


User: Can you recommend a book for me?

BookChat: Sure, do you want a recommendation by title or genre? (Enter 'Title' / 'Genre')

BookChat: Can you type the name of the book you want suggestions for?


In [17]:
recommendation_list = books.show_recommendations(*recommend_question)


BookChat: Here are some books you might like: 
1. For Whom the Bell Tolls
2. To Have and Have Not
3. War and Peace
4. The Snows of Kilimanjaro
5. The Old Man and the Sea
6. The Old Man and the Sea
7. The Capital of the World
8. The Light of the World
9. The Iliad
10. The Killers.


In [18]:
recommendation_choice = books.input("Would you like to know more about a particular book? (Enter 1-10)")

books.query_recommendations(recommendation_list, recommendation_choice)


BookChat: Would you like to know more about a particular book? (Enter 1-10)

BookChat: Let me check the inventory for Book #5...

BookChat: We currently have 20 copies of The Old Man and the Sea in stock.

User: What is the price of that book?

BookChat: Let me find that information for you...

BookChat: The Old Man and the Sea is priced at $8.99.


### Scenario 3: Store Information Inquiry

In [19]:
store.store_location()


User: Where is your store located?

BookChat: We are located at 1841-A Saint-Catherine St W, Montreal, QC H3H 1M2.


In [20]:
store.store_hours()


User: What are your store hours?

BookChat: Our store is open from 11 AM to 7 PM from Monday to Friday. On Saturdays and Sundays, we are open from 12 PM to 5 PM.


In [21]:
store.areas_served()


User: What areas do you serve?

BookChat: We have only one store in Montreal, but we ship anywhere in the Island of Montreal, as well as Laval and South Store.


In [22]:
store.delivery()


User: Do you deliver books?

BookChat: Yes, we do deliver books in the Island of Montreal. Local delivery is a flat rate of $8.50 (plus tax), but it's free for any purchase of $100 or more.


### Scenario 5: Handling Customer Feedback

In [24]:
orders.user_msg("I would like a replacement for my order.")
orders.request_replacement(customerInfo)


User: I would like a replacement for my order.

BookChat: Sorry to hear that, Lakshya. I can help you with that.

BookChat: Your order numbers are: 21

BookChat: Can you please provide your order ID?

BookChat: Thank you for your request.

Order #21, "A Farewell to Arms" will be replaced within 2-3 working days.


In [25]:
orders.user_msg("I would like to provide a review for my order.")
orders.add_review(customerInfo)


User: I would like to provide a review for my order.

BookChat: Thank you, Lakshya. Which order would you like to provide feedback for?

BookChat: Your order numbers are: 21

BookChat: Can you please provide your order ID?

BookChat: Sorry, it seems that you have entered an invalid order ID.

BookChat: Thank you, Lakshya. Which order would you like to provide feedback for?

BookChat: Your order numbers are: 21

BookChat: Can you please provide your order ID?

BookChat: How would you rate the book "A Farewell to Arms"? (on a scale of 1-5)

BookChat: How was your experience with the book?

BookChat: Thank you for your review, Lakshya.


In [26]:
orders.process_reviews()


BookChat: Processing reviews...

BookChat: Here are the reviews with their sentiments:


,reviewID,reviewRating,reviewText,reviewSentiment
0,1,5,"Great book, really enjoyed it!",positive
1,2,4,"Interesting read, a bit slow in the middle.",negative
2,3,3,Not my favorite from this author.,positive
3,4,5,Loved it! Can't wait for the sequel.,negative
4,5,4,"Good book, but the ending was predictable.",negative
5,6,2,"Didn't really like it, the characters were flat.",negative
6,7,5,"Amazing book, couldn't put it down!",positive
7,8,3,"It was okay, but I've read better.",negative
8,9,4,"Really liked it, but the start was a bit slow.",positive
9,10,5,One of the best books I've read this year!,positive
